## Module submission header
### Submission preparation instructions 
_Completion of this header is mandatory, subject to a 2-point deduction to the assignment._ Only add plain text in the designated areas, i.e., replacing the relevant 'NA's. You must fill out all group member Names and Drexel email addresses in the below markdown list, under header __Module submission group__. It is required to fill out descriptive notes pertaining to any tutoring support received in the completion of this submission under the __Additional submission comments__ section at the bottom of the header. If no tutoring support was received, leave NA in place. You may as well list other optional comments pertaining to the submission at bottom. _Any distruption of this header's formatting will make your group liable to the 2-point deduction._

### Module submission group
- Group member 1
    - Name: Luke Hill
    - Email: lh967@drexel.edu

### Additional submission comments
- Tutoring support received: NA
- Other (other): NA

# Assignment Group 4

## Module A _(46 points)_
For this assignment your work will set up a toolkit for plain text enhancement and summarization that can be reused later. Specifically, you'll use an API service known as 'Wikifier' which embeds hyperlink references to encyclopedia articles.

Throughout this problem, we're going to be working with the Wikifier API, which is "a web service that takes a text document as input and annotates it with links to relevant Wikipedia concepts". So, before we can begin with the actual work, it's necessary to request access to this API at the following link: http://wikifier.org/info.html. Read this documentation and familiarize yourself with it, and make sure to register for a `userKey`.
Store your `userKey` in the below cell:

In [2]:
import requests
USER_KEY = "dvkmwpvrlkrccgioyxbcefudeonxub"

__A1.__ _(7 points)_ We're going to make requests to Wikifier. For this, complete the below function that takes a block of text and Wikifier API `KEY` as input, and outputs the JSON response given by the Wikification API (per the following specifications) interpreted as a Python object (deserialized). 

__Important__: For full credit you must read the Wikifier docs: http://wikifier.org/info.html and determine how to build a URL that sets the `'applyPageRankSqThreshold'` field to `True` whenever the `threshold` parameter is non-zero, which should be used to parameterize the corresponding `'pageRankSqThreshold'` field alongside the `language` parameter (for the `'lang'` field), and `KEY` for the `'userKey'` field and `text` for `'text'`.

In [3]:
# A1:Function(7/7)

def get_wikifier_response(text, KEY, threshold = 0.8, language = 'en'):
    
    url = f"http://www.wikifier.org/annotate-article?text={text}&lang={language}&userKey={KEY}&pageRankSqThreshold={threshold}"

    # Making the GET request
    response = requests.get(url)

    return(response.json())

For reference, your output could be:
```
(dict_keys(['title', 'url', 'lang', 'pageRank', 'cosine', 'secLang', 'secTitle', 'secUrl', 'wikiDataItemId', 'wikiDataClasses', 'dbPediaTypes', 'dbPediaIri', 'supportLen', 'support']),
 [([(0, 23)],
   'Colonization of the Moon',
   'http://en.wikipedia.org/wiki/Colonization_of_the_Moon'),
  ([(16, 23),
    (20, 23),
    (128, 138),
    (131, 138),
    (135, 138),
    (289, 296),
    (293, 296)],
   'Moon',
   'http://en.wikipedia.org/wiki/Moon')])
```

In [4]:
# A1:SanityCheck 

test_wiki = get_wikifier_response("".join(["Colonization of the Moon is a concept ",
                                           "employed by some proposals of establishing ",
                                           "permanent human settlement or robotic presence on ",
                                           "the Moon, the closest astronomical body to Earth, ",
                                           "and the Earth's only natural satellite.",
                                           "For a first permanent human space colony ",
                                           "or settlement the choice of the Moon ",
                                           "would benefit from its proximity to Earth."]), 
                                  USER_KEY)
(test_wiki['annotations'][0].keys(), 
 [([(sup['chFrom'], sup['chTo']) for sup in z['support']], 
   z['title'], z['url']) for z in test_wiki['annotations']][:2])

(dict_keys(['title', 'url', 'lang', 'pageRank', 'cosine', 'secLang', 'secTitle', 'secUrl', 'wikiDataItemId', 'wikiDataClasses', 'dbPediaTypes', 'dbPediaIri', 'supportLen', 'support']),
 [([(0, 11), (248, 259)],
   'Space colonization',
   'http://en.wikipedia.org/wiki/Space_colonization'),
  ([(0, 14)],
   'Spanish colonization of the Americas',
   'http://en.wikipedia.org/wiki/Spanish_colonization_of_the_Americas')])

__A2.__ _(7 points)_ Now, supposing we wanted to keep track of this text's information alongside others, we'd first want to make sure our data access is simple and reusable, and don't want to cause any server problems for Wikifier. So, write a function called `get_wikification(text_file_path, KEY)` that accepts a `"path_to/text_file.txt"` (assuming a `'.txt'` extension) and a Wikifier `userKey`.

Your code should load any target text and apply the `get_wikifier_response()` function from __A1__. Make sure your code checks for the availability of a previously-accessed response at the corresponding location: `"path_to/wiki_file.json"` (modified to a `'.json'` extension), which should just be re-loaded from your most recent call to the API, and flushed (deleted) manually if/when your code in __A1__ is at any point revised. 

Note: aside from specifying the `text` and `KEY`, your code should leave all other `get_wikifier_response` parameters as their defaults to meet the target output.

In [7]:
# A2:Function(7/7)
import json, re, os
def get_wikification(text_file_path, KEY):
    ## first, you must convert the provided text path into
    ## the specified path for a .json object response from Wikifier
    json_file_path = re.sub(r'\.txt$', '_wikifier_response.json', text_file_path)

    # check to see if the text's file exists
    if os.path.exists(json_file_path):
        # load any pre-existing data
        wiki = json.load(open(json_file_path, 'r', encoding='utf-8'))
    else:
        ## load the text and use this with your function from A1
        ## to get the wikifier response and store the content on disk
        with open(text_file_path, 'r', encoding='utf-8') as text_file:
            text = text_file.read()

        # Call the get_wikifier_response function
        wikifier_response = get_wikifier_response(text, KEY)

        # Save the response to a JSON file for future use
        with open(json_file_path, 'w', encoding='utf-8') as cached_file:
            json.dump(wikifier_response, cached_file, ensure_ascii=False, indent=2)

        wiki = wikifier_response

    return wiki

For reference, your output could be:
```
[(79, 'World War I', 'http://en.wikipedia.org/wiki/World_War_I'),
 (5, 'Latin', 'http://en.wikipedia.org/wiki/Latin'),
 (7, 'Europe', 'http://en.wikipedia.org/wiki/Europe'),
 (2, 'Nuclear weapon', 'http://en.wikipedia.org/wiki/Nuclear_weapon'),
 (32, 'Washington, D.C.', 'http://en.wikipedia.org/wiki/Washington,_D%2eC%2e')]
```

In [8]:
# A2:SanityCheck

unknown_file_path = "./data/unknown.txt"
unknown_wiki = get_wikification(unknown_file_path, USER_KEY)

[(len(z['support']), z['title'], z['url']) for z in unknown_wiki['annotations']][:5]

[(70, 'Nantes', 'http://en.wikipedia.org/wiki/Nantes'),
 (5, 'Latin', 'http://en.wikipedia.org/wiki/Latin'),
 (3, 'Michael Quatro', 'http://en.wikipedia.org/wiki/Michael_Quatro'),
 (1, 'Certiorari', 'http://en.wikipedia.org/wiki/Certiorari'),
 (1,
  'Attention (machine learning)',
  'http://en.wikipedia.org/wiki/Attention_(machine_learning)')]

__A3.__ _(6 points)_ Now, to work with the Wikifier output we'll need to go through a few steps. The annotations (embedded links) provided by Wikifer reference specific document indices, i.e., for words (and their tokenization) and for characters. We'll work on the character level because it's more precise. To get this off of the ground, make a function called `build_doc(wiki)` that takes a Wikifier output (`wiki`) and builds up the original document from the `wiki['words']` and `wiki['spaces']` fields. 

\[__Hint.__ These two fields alternate,  with `wiki['spaces']` having exactly one more element than `wiki['words']`, i.e., a rebuilt document will join the elements of the two with the first element coming from `wiki['spaces']`\].

In [14]:
def build_doc(wiki):
    doc = []
    
    # Iterate through words and spaces to reconstruct the document
    min_len = min(len(wiki['spaces']), len(wiki['words']))
    
    for i in range(min_len):
        doc.append(wiki['spaces'][i] + wiki['words'][i])

    # Add the remaining words if any
    doc.extend(wiki['words'][min_len:])

    doc = "".join(doc)

    return doc



For reference, your output should be:
```
Mannie, at Mike's request, attends an anti-Authority meeting with a hidden recorder. 
```

In [15]:
# A3:SanityCheck

doc = build_doc(unknown_wiki)
print(doc[:85])

Mannie, at Mike's request, attends an anti-Authority meeting with a hidden recorder. 


__A4.__ _(6 points)_ Now that we can get the document back together we'll also want to collect the links to insert them as annotations. So complete the below function called `get_links(wiki)` that processes the `annotation` objects from the `wiki['annotations']` field (a list of `annotation` objects), and creates/outputs a data object called links, which is a list of tuples:
```
links = [(chFrom, chTo, hyperlink), ...]
```
Hyperlinks will come from `annotation["url"]`, and the `chFrom` and `chTo` elements represent character ranges corresponding to embeddings of `annotation["url"]` in the document. Note that there are multiple character ranges: one for each `support` object in `annotation["support"]` (also a list).  Specifically, each `support` object in the annotation field has two character indices keyed as `support["chFrom"]` and `support["chTo"]` which designate a range (in character indices) over which the hyperlink might well be embedded. 

In [16]:
def get_links(wiki):
    links = []

    # Iterate through each annotation in the 'annotations' field
    for annotation in wiki['annotations']:
        # Iterate through each support object in the 'support' field of the annotation
        for support in annotation['support']:
            # Extract character indices and hyperlink
            ch_from = support['chFrom']
            ch_to = support['chTo']
            hyperlink = annotation['url']

            # Add the information as a tuple to the 'links' list
            links.append((ch_from, ch_to, hyperlink))

    return links

For reference, your output could be:
```
(382,
 [(0, (6, 6, 'http://en.wikipedia.org/wiki/World_War_I')),
  (100, (1092, 1092, 'http://en.wikipedia.org/wiki/Washington,_D%2eC%2e')),
  (200, (3867, 3868, 'http://en.wikipedia.org/wiki/COVID-19_pandemic')),
  (300, (3575, 3581, 'http://en.wikipedia.org/wiki/Public_opinion'))])
```

In [17]:
# A4:SanityCheck

links = get_links(unknown_wiki)
len(links), [(i, link) for i, link in enumerate(links) if not i % 100][:25]

(1013,
 [(0, (6, 6, 'http://en.wikipedia.org/wiki/Nantes')),
  (100, (2177, 2177, 'http://en.wikipedia.org/wiki/Year')),
  (200, (2568, 2570, 'http://en.wikipedia.org/wiki/COVID-19_pandemic')),
  (300, (548, 550, 'http://en.wikipedia.org/wiki/Gleichschaltung')),
  (400, (337, 341, 'http://en.wikipedia.org/wiki/Wheat')),
  (500,
   (2802, 2803, 'http://en.wikipedia.org/wiki/Jin_dynasty_(266%e2%80%93420)')),
  (600, (1014, 1021, 'http://en.wikipedia.org/wiki/Sexual_intercourse')),
  (700, (1733, 1748, 'http://en.wikipedia.org/wiki/Armenia')),
  (800, (2530, 2532, 'http://en.wikipedia.org/wiki/OECD')),
  (900, (3613, 3615, 'http://en.wikipedia.org/wiki/Archaeology')),
  (1000,
   (4832, 4838, 'http://en.wikipedia.org/wiki/Drop_It_Like_It%27s_Hot'))])

__A5.__ _(5 points)_ Your next job is to complete the `embed_link(doc, link)` function that takes the full document output from __A3__, and, a given link (list element) from __A4__'s output, produces an updated document with the hyperlink embedded across the specified character span.

In [20]:
def embed_link(doc, link):
    # Extract information from the link
    ch_from, ch_to, hyperlink = link

    # Embed the hyperlink into the document
    embedded_doc = doc[:ch_from] + f'<a href="{hyperlink}">' + doc[ch_from:ch_to] + '</a>' + doc[ch_to:]

    return embedded_doc


For reference, your output should be:
```html
Mannie, at Mike's request, attends an anti-Authority meeting with a hidden recorder. When police raid the gathering, Mannie flees with <a href="http://en.wikipedia.org/wiki/Wyoming">Wyoming</a>
```

In [21]:
# A5:SanityCheck

for i, link in enumerate(links):
    if link[2] == "http://en.wikipedia.org/wiki/Wyoming":
        annotated_doc = embed_link(doc[:142], link)
        break
print(annotated_doc)

Mannie, at Mike's request, attends an anti-Authority meeting with a hidden recorder. When police raid the gathering, Mannie flees with <a href="http://en.wikipedia.org/wiki/Wyoming">Wyomin</a>g


__A6.__ _(4 points)_ If you haven't yet noticed, there are many link predictions which point to the same page. Embedding all of these together in our document not only won't be informative but simply won't be possible. As a simple first heuristic, complete the function below to filter the links we get from a given `wiki` so that only one embedding (text span) is maintained for each link, _corresponding to the longest span over which they were embedded_. Your job is to track these in the `longest_links` dictionary object, which should be keyed by the URLs, and have values equal to the list-rows returned by the `get_links` function. 

Note: since `sorted(longest_links.values())` is returned, your function necessarily returns a list where each URL only appears once.

In [23]:
# A6:Function(4/4)

def filter_longest_links(wiki):
    links = get_links(wiki)
    longest_links = {}

    # Iterate through each link
    for link in links:
        url = link[2]  # Extract the URL

        # Check if the URL is already in the dictionary
        if url not in longest_links:
            longest_links[url] = link
        else:
            # Compare the character spans and keep the longer one
            existing_link = longest_links[url]
            if link[1] - link[0] > existing_link[1] - existing_link[0]:
                longest_links[url] = link

    return sorted(longest_links.values())


For reference, your output could be:
```
[(8, 9, 'http://en.wikipedia.org/wiki/Latin'),
 (35, 36, 'http://en.wikipedia.org/wiki/Europe'),
 (83, 83, 'http://en.wikipedia.org/wiki/Washington,_D%2eC%2e'),
 (102, 104, 'http://en.wikipedia.org/wiki/COVID-19_pandemic'),
 (135, 141, 'http://en.wikipedia.org/wiki/Wyoming'),
 (255, 263, 'http://en.wikipedia.org/wiki/Professor'),
 (255, 282, 'http://en.wikipedia.org/wiki/The_Moon_Is_a_Harsh_Mistress'),
 (326, 335, 'http://en.wikipedia.org/wiki/Hydroponics'),
 (337, 341, 'http://en.wikipedia.org/wiki/Wheat'),
 (355, 357, 'http://en.wikipedia.org/wiki/World_War_I')]
```

In [24]:
# A6:SanityCheck

longest_links = filter_longest_links(unknown_wiki)
longest_links[:10]

[(6, 6, 'http://en.wikipedia.org/wiki/Nantes'),
 (8, 9, 'http://en.wikipedia.org/wiki/Latin'),
 (11, 16, 'http://en.wikipedia.org/wiki/Michael_Quatro'),
 (18, 24, 'http://en.wikipedia.org/wiki/Certiorari'),
 (27, 33, 'http://en.wikipedia.org/wiki/Attention_(machine_learning)'),
 (35, 36, 'http://en.wikipedia.org/wiki/Europe'),
 (53, 59, 'http://en.wikipedia.org/wiki/Freedom_of_assembly'),
 (61, 64, 'http://en.wikipedia.org/wiki/Two-round_system'),
 (66, 66, 'http://en.wikipedia.org/wiki/Year'),
 (68, 73, 'http://en.wikipedia.org/wiki/Camouflage')]

__A7.__ _(6 points)_ The responses received from Wikifier are really only _predictions_ of where links should go. This means that multiple link predictions might span overlapping portions of source text. So, we'll have to make sure our code does some light decision making to avoid overlapping hyperlinks and broken html code. 

Your job now is to complete the `embed_links()` function, which will manage the link embedding process, particularly, the overlapping link problem. Specifically, your function _must_:

1. Take a wikifier, `wiki`, response as input.
2. Build the document using `build_doc()` and store its output as a string called `doc`
3. Apply `get_links()` function, storing the `links` output 
4. _Reverse_ sort the `link`  by their ending points, i.e., `support["chTo"]` values
5. Loop through the _sorted_ `links` 
6. Conditionally use the `embed_link()` function to embed links into `doc` in the loop. Specifically, only embed a link if its `support["chTo"]` value is less than the previous one's `support["chFrom"]` value, or if the link to embed is the first in the loop.
7. `return` the link-enhanced `doc`.

In [25]:
# A7:Function(6/6)

def embed_links(wiki):
    ## build the document, gather its links
    ## and store only the longest embedding for each (in characters)
    doc = build_doc(wiki)
    links = get_links(wiki)        
    longest_links = filter_longest_links(wiki)
    ## sort the links from high to low by their second 'column'
    sorted_links = sorted(longest_links, key=lambda x: x[1], reverse=True)

    ## you must keep track of the previously-entered link's starting index
    prev_from = float("Inf")
    
    for link in sorted_links:
        ## only embed if the current link doesn't overlap 'into'
        ## the previously-entered link's starting index
        
        # Extract information from the link
        ch_from, ch_to, hyperlink = link

        # Conditionally embed the link into the document
        if ch_to < prev_from or not sorted_links.index(link):
            doc = embed_link(doc, link)
            prev_from = ch_from

    return doc

For reference, your first paragraph's rendering should come close to the below:

Mannie, at Mike's request, attends [an](http://en.wikipedia.org/wiki/Europe) anti-Authority meeting with a hidden recorder[.](http://en.wikipedia.org/wiki/Washington,_D%2eC%2e) When police raid [the](http://en.wikipedia.org/wiki/COVID-19_pandemic) gathering, Mannie flees with [Wyoming](http://en.wikipedia.org/wiki/Wyoming) ("Wyoh") Knott, a political agitator, whom he introduces to Mike. They meet Mannie's former mentor, the elderly [](http://en.wikipedia.org/wiki/Professor)[Professor Bernardo de la Paz](</a>http://en.wikipedia.org/wiki/The_Moon_Is_a_Harsh_Mistress), who claims that Luna must stop exporting [hydroponic](http://en.wikipedia.org/wiki/Hydroponics) [grain](http://en.wikipedia.org/wiki/Wheat) to Earth or [its](http://en.wikipedia.org/wiki/World_War_I) ice-mines will soon be exhausted, leaving the Moon waterless. Joining the [cabal](http://en.wikipedia.org/wiki/Cabal), Mike calculates that without prevention, food riots [will occur](http://en.wikipedia.org/wiki/Gulf_War) in seven years and [cannibalism](http://en.wikipedia.org/wiki/Cannibalism) in nine. Wyoh and the Professor decide to start a revolution, and persuade Mannie to join after Mike calculates a one in seven chance of success.

In [26]:
# A7:SanityCheck

from IPython.core.display import HTML

annotated_doc = embed_links(unknown_wiki)
annotated_doc = re.sub("\n", "<br>", annotated_doc)
HTML(annotated_doc)

__A8.__ _(5 pts)_ It's time to package your code up so someone else can use it! Specifically, your goal here will be to place _all of the functions_ that you defined within this module into a file called `./wikifier.py`; doing so will technically create our very own _Python module_. When this is complete, you'll be able to operate your code's function by loading the module, i.e., `import wikifier`, and executing the two controller methods:

- `wikification = wikifier.get_wikification("example_text.txt", USER_KEY)`
- `annotated_doc = wikifier.embed_links(wikification)`

Note: Your `./wikifier.py` Python module must also contain any necessary module loads, and since it is being distributed for _others_ to use it absolutely must use relative file paths, assuming the `./data/` location for application data. 

Importantly, to write the contents of a cell to a file, your job is to utilize the `inspect` module, whose `inspect.getsource()` method may be applied to a function to get its source as a  string. Once as strings, you must concatenate the functions together&mdash;onto the module loads&mdash;while being sure to separate each function by two newlines (`"\n\n"`), for clarity. Note: for convenience, all of your functions are maintained in the `all_function` list object below.

In [30]:
# A8:Inline(5/5)

import inspect
import os, re, json, requests

def get_wikifier_response(text, KEY, threshold=0.8, language='en'):
    url = f"http://www.wikifier.org/annotate-article?text={text}&lang={language}&userKey={KEY}&pageRankSqThreshold={threshold}"
    response = requests.get(url)
    return response.json()

def get_wikification(text_file_path, KEY):
    json_file_path = re.sub(r'\.txt$', '_wikifier_response.json', text_file_path)

    if os.path.exists(json_file_path):
        with open(json_file_path, 'r', encoding='utf-8') as cached_file:
            return json.load(cached_file)
    else:
        with open(text_file_path, 'r', encoding='utf-8') as text_file:
            text = text_file.read()
        wikifier_response = get_wikifier_response(text, KEY)
        with open(json_file_path, 'w', encoding='utf-8') as cached_file:
            json.dump(wikifier_response, cached_file, ensure_ascii=False, indent=2)
        return wikifier_response

def build_doc(wiki):
    words = wiki['words']
    spaces = wiki['spaces']
    document = ""
    
    # Ensure the lengths of spaces and words are the same
    min_len = min(len(spaces), len(words))
    
    for i in range(min_len):
        document += spaces[i] + words[i]

    # Add the remaining words if any
    document += "".join(words[min_len:])

    return document

def get_links(wiki):
    links = []
    for annotation in wiki['annotations']:
        for support in annotation['support']:
            ch_from = support['chFrom']
            ch_to = support['chTo']
            hyperlink = annotation['url']
            links.append((ch_from, ch_to, hyperlink))
    return links

def embed_link(doc, link):
    ch_from, ch_to, hyperlink = link
    embedded_doc = doc[:ch_from] + f'<a href="{hyperlink}">' + doc[ch_from:ch_to] + '</a>' + doc[ch_to:]
    return embedded_doc

def filter_longest_links(wiki):
    links = get_links(wiki)
    longest_links = {}
    for link in links:
        url = link[2]
        if url not in longest_links:
            longest_links[url] = link
        else:
            existing_link = longest_links[url]
            if link[1] - link[0] > existing_link[1] - existing_link[0]:
                longest_links[url] = link
    return sorted(longest_links.values())

def embed_links(wiki):
    doc = build_doc(wiki)
    links = get_links(wiki)
    longest_links = filter_longest_links(wiki)
    sorted_links = sorted(longest_links, key=lambda x: x[1], reverse=True)
    prev_from = float("Inf")
    for link in sorted_links:
        ch_from, ch_to, hyperlink = link
        if ch_to < prev_from or not sorted_links.index(link):
            doc = embed_link(doc, link)
            prev_from = ch_from
    return doc

#---your code starts here---
all_functions = [get_wikifier_response, get_wikification, build_doc, 
                 get_links, embed_link, filter_longest_links, embed_links]

module_script = "import os, re, json, requests\n\n"
for function in all_functions:
    module_script += inspect.getsource(function) + "\n\n"

with open('wikifier.py', 'w') as f:
    f.write(module_script)
#---your code stops here---

print(module_script[:150])


import os, re, json, requests

def get_wikifier_response(text, KEY, threshold=0.8, language='en'):
    url = f"http://www.wikifier.org/annotate-articl


For reference, your output from the above should be identical to the text of the functions you've authored. Likewise, the below `SanityCheck` should produce the exact same output as __A7.__ Also, don't forget&mdash;TANSTAAFL!

In [31]:
# A8:SanityCheck

import wikifier, importlib
importlib.reload(wikifier)

wikification = wikifier.get_wikification(unknown_file_path, USER_KEY)
annotated_doc = wikifier.embed_links(wikification)
HTML(re.sub("\n", "<br>", annotated_doc))